In [122]:
import pandas as pd

import plotly.express as px

import warnings

warnings.simplefilter('ignore')


In [123]:
gastos = pd.read_csv(
    'https://www.gov.br/secretariageral/pt-br/acesso-a-informacao/informacoes-classificadas-e-desclassificadas/Planilha12003a2022.csv',
    encoding='latin1', delimiter=';')


In [124]:
gastos_processed = gastos.copy()


In [125]:
gastos_processed.loc[:, 'DATA PGTO'] = pd.to_datetime(gastos_processed['DATA PGTO'], errors='coerce', format="%d/%m/%Y")

gastos_processed.loc[:, 'ANO'] = gastos_processed['DATA PGTO'].dt.year


gastos_processed = gastos_processed.dropna(axis=0, subset=['CPF SERVIDOR'])


gastos_processed.loc[:, 'VALOR_NUM'] = pd.to_numeric(gastos_processed['VALOR'].str.replace(
    r'R\$', '').str.replace(".", '').str.replace(',', '.').str.strip(), errors='coerce')

gastos_processed = gastos_processed.dropna(subset=['VALOR_NUM'])


In [126]:
def presidente(ano):

    if ano <= 2010:
        return 'LULA'
    elif ano <= 2016:
        return 'DILMA'
    elif ano <= 2018:
        return "TEMER"
    elif ano <= 2022:
        return "BOLSONARO"


gastos_processed['PRES'] = gastos_processed['ANO'].apply(presidente)


In [127]:
gastos_processed = gastos_processed.sort_values("ANO")


In [128]:
gastos_presidente = gastos_processed.groupby(["ANO", "PRES"]).agg({"VALOR_NUM": sum}).reset_index()


In [130]:
fig = px.bar(gastos_presidente, x='ANO', y="VALOR_NUM", color='PRES',
             title="Gastos do cartão presidencial por presidente",
             labels={"VALOR_NUM": "Valor em milhões", 'PRES': "PRESIDENTE"})

fig = fig.update_xaxes(tickangle=45, tickvals=gastos_presidente['ANO'][::])


In [132]:
fig.show()
